In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import json
import pytz
import plotly.express as px
import os
import time

In [2]:
m_txt = ["Gener","Febrer","Marc","Abril","Maig","Juny","Juliol","Agost","Setembre","Octubre","Novembre","Desembre"]

In [4]:
def extreu_datetime(num):
    dt = utc.localize(datetime.datetime.utcfromtimestamp(num))
    dt = dt.astimezone(bcn_tz)
    return dt

def weight_obs(r):
    w = 0
    if r["new_station"] or r["l_lr"] > r["dt"].minute*60+r["dt"].second:
        w += r["dt"].minute*60+r["dt"].second
    else:
        w += r["l_lr"]/2
    if r["next_station"] or r["n_lr"] > 3600-(r["dt"].minute*60+r["dt"].second):
        w += 3600-(r["dt"].minute*60+r["dt"].second)
    else:
        w += r["n_lr"]/2
    return w/3600

for a in range(2019, 2023):
    for m in range(1, 13):
        ini = datetime.datetime.now()
        if 202006 <= a*100+m <= 202211:
            carpeta = "{}_{:02d}_{}_BicingNou_ESTACIONS".format(a, m, m_txt[m-1])
            arxiu = os.listdir("{}_{:02d}_{}_BicingNou_ESTACIONS".format(a, m, m_txt[m-1]))[0]
            df = pd.read_csv(
                    carpeta+"/"+arxiu,
                    usecols=["station_id", "num_bikes_available","num_bikes_available_types.mechanical",
                             "num_bikes_available_types.ebike","num_docks_available","last_reported",
                             "is_renting","is_returning"],
                    dtype={"station_id":"Int16","num_bikes_available":"Int16","num_bikes_available_types.mechanical":"Int16",
                             "num_bikes_available_types.ebike":"Int16","num_docks_available":"Int16","last_reported":"Int32",
                             "is_renting":"Int8","is_returning":"Int8"})
            print(a*100+m, datetime.datetime.now()-ini)
            df = df.sort_values(["station_id","last_reported"])
            df.drop_duplicates(ignore_index=True)
            utc = pytz.timezone("UTC")
            bcn_tz = pytz.timezone("Europe/Madrid")
            df["dt"] = df["last_reported"].apply(extreu_datetime)
            df = df[df.dt.apply(lambda x: x.month == m)].reset_index(drop=True)
            df["new_station"] = df["station_id"].diff().fillna(1).astype("bool")
            df["next_station"] = df["station_id"].diff(periods=-1).fillna(1).astype("bool")
            df["l_lr"] = df["last_reported"].diff().fillna(3600).astype("Int32")
            df["n_lr"] = -df["last_reported"].diff(periods=-1).fillna(3600).astype("Int32")
            print(a*100+m, datetime.datetime.now()-ini)
            df["hour"] = df["dt"].apply(lambda x: x.hour).astype("Int8")
            df["minute"] = df["dt"].apply(lambda x: x.minute).astype("Int8")
            df["second"] = df["dt"].apply(lambda x: x.second).astype("Int8")
            df["year"] = df["dt"].apply(lambda x: x.year).astype("Int32")
            df["month"] = df["dt"].apply(lambda x: x.month).astype("Int8")
            df["day"] = df["dt"].apply(lambda x: x.day).astype("Int8")
            df["weekday"] = df["dt"].apply(lambda x: x.weekday()).astype("Int8")
            print(a*100+m, datetime.datetime.now()-ini)
            df["ocupacio"] = 0.
            df.loc[(df["num_bikes_available"]+df["num_docks_available"])>0,"ocupacio"] = (df["is_renting"] * df["num_bikes_available"])/(df["num_bikes_available"]+df["num_docks_available"])
            df["plena"] = 0.
            df.loc[(df["is_returning"] == 0) | (df["num_docks_available"] == 0),"plena"] = 1
            df["buida"] = 0.
            df.loc[(df["is_renting"] == 0) | (df["num_bikes_available"] == 0),"buida"] = 1
            print(a*100+m, datetime.datetime.now()-ini)
            df["weight"] = 0.
            c1 = (df["new_station"]) | (df["l_lr"] >  df["minute"]*60+df["second"])
            df.loc[c1,"weight"] = (df["weight"] + df["minute"]*60+df["second"])
            df.loc[~c1,"weight"] = (df["weight"] + df["l_lr"]/2)
            c2 = (df["next_station"]) | (df["n_lr"] >=  3600-(df["minute"]*60+df["second"]))
            df.loc[c2,"weight"] = (df["weight"] + 3600-(df["minute"]*60+df["second"]))
            df.loc[~c2,"weight"] = (df["weight"] + df["n_lr"]/2)
            df["weight"] = df["weight"]/3600
            print(a*100+m, datetime.datetime.now()-ini)
            df["w_bike"] = df["num_bikes_available"]*df["weight"]
            df["w_buida"] = df["buida"]*df["weight"]
            df["w_plena"] = df["plena"]*df["weight"]
            df["w_ocupacio"] = df["ocupacio"]*df["weight"]
            agg_hora_station = df.groupby(["station_id","year","month","weekday","hour"]).agg(
                avg_bikes=pd.NamedAgg(column="w_bike", aggfunc="sum"),
                full=pd.NamedAgg(column="w_buida", aggfunc="sum"),
                empty=pd.NamedAgg(column="w_plena", aggfunc="sum"),
                ocupacio=pd.NamedAgg(column="w_ocupacio", aggfunc="sum"),
                weight=pd.NamedAgg(column="weight", aggfunc="sum")
            )
            agg_hora_station.loc[:,"avg_bikes"] = agg_hora_station["avg_bikes"] / agg_hora_station["weight"]
            agg_hora_station.loc[:,"full"] = agg_hora_station["full"] / agg_hora_station["weight"]
            agg_hora_station.loc[:,"empty"] = agg_hora_station["empty"] / agg_hora_station["weight"]
            agg_hora_station.loc[:,"ocupacio"] = agg_hora_station["ocupacio"] / agg_hora_station["weight"]
            agg_hora_station.to_csv("OUTPUTS/stats_{}_{:02d}.csv".format(a, m))

202006 0:00:17.673979
202006 0:00:44.712162
202006 0:00:56.547242
202006 0:00:56.945609
202006 0:00:57.636787
202007 0:00:18.527026
202007 0:00:47.172713
202007 0:00:59.498003
202007 0:00:59.912251
202007 0:01:00.683773
202008 0:00:17.268063
202008 0:00:44.811397
202008 0:00:56.452582
202008 0:00:56.797539
202008 0:00:57.504184
202009 0:00:24.177236
202009 0:00:58.746435
202009 0:01:12.287115
202009 0:01:12.704305
202009 0:01:13.514425
202010 0:00:21.504304
202010 0:00:53.048412
202010 0:01:06.108046
202010 0:01:06.491906
202010 0:01:07.277293
202011 0:00:17.001317
202011 0:00:42.837858
202011 0:00:54.754143
202011 0:00:55.073378
202011 0:00:55.775028
202012 0:00:21.232944
202012 0:00:53.799103
202012 0:01:08.010966
202012 0:01:08.767790
202012 0:01:10.138105
202101 0:00:22.658121
202101 0:00:55.212640
202101 0:01:09.282020
202101 0:01:09.656210
202101 0:01:10.482480
202102 0:00:19.294784
202102 0:00:49.363284
202102 0:01:01.663436
202102 0:01:02.016044
202102 0:01:02.984487
202103 0:0

In [19]:
info = pd.read_csv("2022_11_Novembre_BicingNou_INFORMACIO/2022_11_Novembre_BicingNou_INFORMACIO.csv",
                   usecols=["station_id", "name", "lat", "lon", "altitude"]
                  ).drop_duplicates(ignore_index=True, subset=["station_id"])
info.to_csv("OUTPUTS/info.csv")